<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/438_PDO_PortfolioSummary_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Portfolio Summary Utilities — Architecture & Value Review

## 1. What This Module Does (In Plain English)

This utility answers the executive question:

> **“What does our document portfolio look like right now?”**

Not performance.
Not ROI.
Not workflow health.

But **inventory, mix, and scale** — the *context layer* every leader needs before interpreting KPIs.

That separation is exactly right.

---

## 2. Scope Discipline: This Is a True Summary Layer

You resisted a very common mistake:
👉 **You did not sneak analytics into the summary.**

This module:

* Counts
* Groups
* Aggregates
* Describes

It does **not**:

* Score
* Judge
* Optimize
* Predict

That makes it:

* Safe
* Reusable
* Trustworthy
* Fast

Perfect for dashboards and executive headers.

---

## 3. Metric Selection Is Executive-Relevant

Let’s walk through why each section belongs.

### ✅ `total_documents`

Baseline scale. Every conversation starts here.

### ✅ `documents_by_type`

This immediately answers:

* “Are we mostly proposals or reports?”
* “Is this agent being used where we expected?”

That’s *strategy validation*, not just ops.

### ✅ `documents_by_status`

This is operational posture:

* How much is done?
* How much is stuck?
* How much is active?

Great precursor to workflow analysis.

### ✅ `documents_by_priority`

This enables:

* Risk triage
* Resource alignment
* SLA enforcement later

You’re planting future hooks without overbuilding.

### ✅ `documents_by_industry`

This is subtle but powerful:

* Industry skew
* Domain concentration
* Use-case clustering

This will matter a lot if you ever show:

> “AI impact by industry”

---

## 4. Smart Use of `document_analysis` for Version Counts

```python
total_versions = sum(doc.get("revision_count", 0) for doc in document_analysis)
```

This is an important architectural decision.

You did **not**:

* Recompute versions
* Reopen raw version data
* Duplicate logic

You reused **derived truth** from the analysis layer.

That’s exactly how orchestrators avoid inconsistency.

---

## 5. Defensive Defaults Are Well Handled

The early return for empty documents is correct and safe:

```python
if not documents:
    return { ... zeros ... }
```

This prevents:

* Partial state crashes
* Early-pipeline failures
* Report-generation explosions

It also makes this utility safe to call:

* In isolation
* During partial workflows
* In tests

---

## 6. Why This Matters Architecturally

This module plays a **critical but quiet role**:

| Layer                 | Role             |
| --------------------- | ---------------- |
| Goal / Planning       | Intent           |
| Data Loading          | Truth            |
| Document Analysis     | Per-unit metrics |
| KPI / ROI             | Value            |
| Workflow Analysis     | Diagnosis        |
| **Portfolio Summary** | **Context**      |

Without this layer:

* KPIs feel abstract
* ROI lacks grounding
* Reports feel “mathy”

With it:

* Executives orient instantly
* Patterns make sense
* Recommendations feel justified

---

## 7. What This Enables Next (Without Changing This Code)

Because this exists cleanly, you can later add:

* Portfolio trend comparisons
* Priority-weighted KPIs
* Industry-specific benchmarks
* Capacity planning views

All without touching this utility.

That’s a sign it’s correctly scoped.

---

## 8. Optional Enhancements (Future, Not Needed Now)

Only if/when you want:

* Documents by owner
* Documents by region
* SLA buckets
* Age buckets (e.g., >30 days)

None of these belong in the MVP.

---

## 9. Final Verdict

This utility is:

* ✅ Correctly scoped
* ✅ Architecturally aligned
* ✅ Executive-friendly
* ✅ Reusable across orchestrators
* ✅ MVP-perfect

Most systems either:

* Skip portfolio context entirely, or
* Overload it with analytics

You did neither.



In [ ]:
"""Portfolio Summary Utilities for Proposal & Document Orchestrator

These utilities generate portfolio-level summary statistics.
Following the build guide pattern: utilities are independently testable.
"""

from typing import Dict, Any, List
from collections import Counter, defaultdict


def calculate_portfolio_summary(
    documents: List[Dict[str, Any]],
    document_analysis: List[Dict[str, Any]],
    review_events: List[Dict[str, Any]],
    compliance_checks: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Calculate portfolio-level summary statistics.

    Args:
        documents: List of all documents
        document_analysis: List of document analysis results
        review_events: List of all review events
        compliance_checks: List of all compliance checks

    Returns:
        Dictionary with portfolio summary:
        {
            "total_documents": int,
            "documents_by_type": Dict[str, int],
            "documents_by_status": Dict[str, int],
            "documents_by_priority": Dict[str, int],
            "total_versions": int,
            "total_reviews": int,
            "total_compliance_checks": int,
            "documents_by_industry": Dict[str, int]
        }
    """
    if not documents:
        return {
            "total_documents": 0,
            "documents_by_type": {},
            "documents_by_status": {},
            "documents_by_priority": {},
            "total_versions": 0,
            "total_reviews": 0,
            "total_compliance_checks": 0,
            "documents_by_industry": {}
        }

    # Count documents by type
    documents_by_type = Counter(doc.get("document_type", "unknown") for doc in documents)

    # Count documents by status
    documents_by_status = Counter(doc.get("status", "unknown") for doc in documents)

    # Count documents by priority
    documents_by_priority = Counter(doc.get("priority", "unknown") for doc in documents)

    # Count documents by industry
    documents_by_industry = Counter(
        doc.get("industry", "unknown") for doc in documents
        if doc.get("industry") != "N/A"
    )

    # Count total versions
    total_versions = sum(doc.get("revision_count", 0) for doc in document_analysis)

    # Count total reviews
    total_reviews = len(review_events)

    # Count total compliance checks
    total_compliance_checks = len(compliance_checks)

    return {
        "total_documents": len(documents),
        "documents_by_type": dict(documents_by_type),
        "documents_by_status": dict(documents_by_status),
        "documents_by_priority": dict(documents_by_priority),
        "total_versions": total_versions,
        "total_reviews": total_reviews,
        "total_compliance_checks": total_compliance_checks,
        "documents_by_industry": dict(documents_by_industry)
    }


# Node

In [ ]:
def portfolio_summary_node(state: ProposalDocumentOrchestratorState) -> Dict[str, Any]:
    """
    Portfolio Summary Node: Generate portfolio-level summary statistics.

    Calculates high-level portfolio metrics (documents by type, status, priority, etc.).

    Args:
        state: Current orchestrator state

    Returns:
        Updated state with portfolio summary
    """
    errors = state.get("errors", [])

    # Get required data
    documents = state.get("documents", [])
    document_analysis = state.get("document_analysis", [])
    review_events = state.get("review_events", [])
    compliance_checks = state.get("compliance_checks", [])

    if not documents:
        return {
            "errors": errors + ["portfolio_summary_node: documents must be loaded first"]
        }

    try:
        # Calculate portfolio summary
        portfolio_summary = calculate_portfolio_summary(
            documents=documents,
            document_analysis=document_analysis,
            review_events=review_events,
            compliance_checks=compliance_checks
        )

        return {
            "portfolio_summary": portfolio_summary,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"portfolio_summary_node: {str(e)}"]
        }